# Functions for (1) aggregration of image collection, (2) band selection based on specific index, (3) mapping functions for time-series

In [40]:
# import gee
import ee
import folium

In [41]:
#authentificate, generate token and allow access rights
ee.Authenticate()
ee.Initialize()

## (1) aggregration of image collection

In [42]:
#function for aggregratrion
def aggregate_image_collection(collection, start_date, end_date, aggregation_method):
    collection = ee.ImageCollection(collection).filterDate(start_date, end_date)
    
    if aggregation_method == 'mean':
        return collection.mean()
    elif aggregation_method == 'median':
        return collection.mean()
    elif aggregation_method == 'sum':
        return collection.mean()
    elif aggregation_method == 'min':
        return collection.mean()
    elif aggregation_method == 'max':
        return collection.mean()
    else:
        print("Wrong aggregation method! Available: mean, median, sum, min, max")

## (2) band selection based on specific index

In [43]:
def select_bands(image, index_name, satellite_name):

    if satellite_name == 'Sentinel-2':
        band_mapping = {
            'RED': 'B4',
            'GREEN': 'B3',
            'BLUE': 'B2',
            'NIR': 'B8'
        }
    elif satellite_name == 'Landsat-8':
        band_mapping = {
            'RED': 'B4',
            'GREEN': 'B3',
            'BLUE': 'B2',
            'NIR': 'B5'
        }
    else:
        print('Wrong satellite! Available: Sentinel-2 or Landsat-8')

    if index_name in ['RED', 'GREEN', 'BLUE', 'NIR']:
        return image.select(band_mapping[index_name])
    else:
        raise ValueError("Unsupported index!")
    

## (3) mapping functions for time-series

In [44]:
'''
def map_time_series(collection, start_date, end_date, index_name):
    collection = ee.ImageCollection(collection).filterDate(start_date, end_date)
    index_time_series = collection.map(calc_ndvi)
    
    return index_time_series
'''

'\ndef map_time_series(collection, start_date, end_date, index_name):\n    collection = ee.ImageCollection(collection).filterDate(start_date, end_date)\n    index_time_series = collection.map(calc_ndvi)\n    \n    return index_time_series\n'

In [45]:
'''
def calc_ndvi(image):
    red = select_bands(image, 'RED', 'Sentinel-2')
    nir = select_bands(image, 'NIR', 'Sentinel-2')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)
'''

"\ndef calc_ndvi(image):\n    red = select_bands(image, 'RED', 'Sentinel-2')\n    nir = select_bands(image, 'NIR', 'Sentinel-2')\n    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')\n    return image.addBands(ndvi)\n"

## Test

In [46]:
# area of interest in olomouc, czechia
aoi = ee.Geometry.Rectangle(17.22,
                            49.61,
                            17.26,
                            49.63)

In [47]:
# set the start and end date
start_date = '2023-01-01'
end_date = '2023-12-31'

In [48]:
# select sentinel 2 imagery
collection = ee.ImageCollection('COPERNICUS/S2').filterBounds(aoi).filterDate(start_date, end_date)
collection = collection.filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20);

In [49]:
# Aggregate the collection to get the mean image
mean_image = aggregate_image_collection(collection, start_date, end_date, 'mean')

In [50]:
# Select the red band of the image
red = select_bands(mean_image, 'RED', 'Sentinel-2')
print(red.getInfo())

{'type': 'Image', 'bands': [{'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [51]:
# Select the nir band of the image
nir = select_bands(mean_image, 'NIR', 'Sentinel-2')
print(nir.getInfo())

{'type': 'Image', 'bands': [{'id': 'B8', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [52]:
'''
# Create a time series of NDVI for Sentinel-2
ndvi_time_series = map_time_series(collection, start_date, end_date, 'NDVI')
'''

"\n# Create a time series of NDVI for Sentinel-2\nndvi_time_series = map_time_series(collection, start_date, end_date, 'NDVI')\n"

In [53]:
'''
print(ndvi_time_series.getInfo())
'''

'\nprint(ndvi_time_series.getInfo())\n'